!! TO DO: understand math behind good cases ratio w.r.t. std/scale_up_factor (currently hyps tuned based on experimental evidence) !!






In [88]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from scipy.special import softmax
import random
import pandas as pd
import json

# hyperparameters
no_puzzlepieces = 15
no_instances = 200000   
std = 0.275 # noise in coordinate confidence                           
std_inst = 0.075 # noise in instruction confidence         
coord = list(range(no_puzzlepieces))
scale_up_factor = 1   
scale_up_factor_inst = 3 # parameter to scale up all values before softmaxing to prevent all values from being all too similar

# storing intermediate and final data
X=[]

X_coord_conf_ideal =[]
X_coord_conf_noisy =[]

X_inst_conf_ideal =[]
X_inst_conf_noisy =[]

X_mov_vec = []

y = []


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Instructions and Instruction Confidence

In [89]:
for m in range(no_instances):
        
    #print('\n')
    #print(f'instance: {m}')
    
    inst_p_ideal = []
    inst_p_noisy = []
    
    for group in range(3):
        
        #print('\n')
        #print(f'group: {group}')
        
        #############################################################################
        ############## SET TRUE INSTRUCTION CONFIDENCE ##############################
        #############################################################################
        
        current_inst_p_ideal = random.randint(0,1)
        #print(current_inst_p_ideal)
        inst_p_ideal.append(current_inst_p_ideal)
        
        
        #############################################################################
        ########################  NORMALLY DISTRIBUTED NOISE ########################
        #############################################################################
        
        # create an auxilary probability that an instruction was not given (1 - inst_p) 
        inst_p_ideal_aux = []
        inst_p_ideal_aux.append(current_inst_p_ideal) # probability that an instruction was given
        if current_inst_p_ideal == 0: # probability that an instruction was not given
            inst_p_ideal_aux.append(1) 
        else:
            inst_p_ideal_aux.append(0) 

        
        # upscale values
        inst_p_aux_upscaled = np.multiply(inst_p_ideal_aux, scale_up_factor_inst) 
        #print(inst_p_aux_upscaled) 
        
        # noise up those ideal probabilities (n) for "instruction was given" and "instruction was not given"
        inst_p_noisy_aux = []
        for n in range(2):
            inst_p_noisy_aux.append(np.random.normal(scale = std_inst, loc = inst_p_aux_upscaled[n], size = 1)[0])                             
        #print(inst_p_noisy_aux)

        # squash values to sum up to one to represent probabilities
        inst_p_noisy_aux_squashed = softmax(inst_p_noisy_aux)
        #print(inst_p_noisy_aux_squashed)
        
        #print(inst_p_noisy_aux_squashed[0])
        inst_p_noisy.append(inst_p_noisy_aux_squashed[0])
    
    # store
    X_inst_conf_ideal.append(inst_p_ideal)
    X_inst_conf_noisy.append(inst_p_noisy)


# Coordinates and Coordinate Confidence

In [90]:
for m in range(no_instances):
    
    
    #print('\n')
    #print(f'instance: {m}')
    

    #############################################################################
    ############## SET TRUE COORDINATE & IDEAL PROBABILITIES ####################
    #############################################################################


    # set all coordinate probabilities p = 0.0
    coordinate_p_ideal = np.zeros(no_puzzlepieces)

    # determine true coordinate and set its probability to p = 1.0
    i = random.randint(0,no_puzzlepieces-1)
    coordinate_p_ideal[i] = 1.0

    # show all probabilities
    #print(coordinate_p_ideal)
    
    # store ideal probabilities
    X_coord_conf_ideal.append(coordinate_p_ideal)


    #############################################################################
    ########################  NORMALLY DISTRIBUTED NOISE ########################
    #############################################################################

    
    # auxilary lists to store different group's outputs per instance
    both_groups = []
    for group in range(2):
        
        if X_inst_conf_ideal[m][group]==0:
            
            # add uniformly distributed noise on ideal probabilities 
            coordinate_p_noisy = []
            for i in range(no_puzzlepieces):
                coordinate_p_noisy.append(np.random.uniform())
            
        else:    

            # add normally distributed noise on ideal probabilities 
            coordinate_p_noisy = []
            for i in range(no_puzzlepieces):
                coordinate_p_noisy.append(np.random.normal(scale = std, loc = coordinate_p_ideal[i], size = 1)[0])

                
        # upscale values
        coordinate_p_noisy_upscaled = np.multiply(coordinate_p_noisy, scale_up_factor)                               

        # squash values to sum up to one to represent probabilities
        coordinate_p_noisy_squashed = softmax(coordinate_p_noisy_upscaled)

        # save group's probability distribution in overall df
        both_groups.append(coordinate_p_noisy_squashed)
        #print(both_groups[group])

    #X.append(both_groups)
    X_coord_conf_noisy.append(both_groups)



# HYPERPARAMETER-TUNING: COORDINATE CONFIDENCE NOISE

---



In [91]:
no_true_inst = 0 # counting how many times an instruction was really given
no_good_pred = 0 # counting right model predictions when really given an instruction

no_two_true_inst = 0 # counting how many times both group1 and group2 really received an instruction
no_two_good_pred = 0 # counting how many times both groups gave the right prediction


for m in range(no_instances):

    # checking correct predictions per group
    for group in range(2):
        if X_inst_conf_ideal[m][group]==1:
            no_true_inst += 1
            if np.argmax(X_coord_conf_ideal[m])==np.argmax(X_coord_conf_noisy[m][group]):
                no_good_pred += 1

    # checking correct predictions for both groups
    if X_inst_conf_ideal[m][0]==X_inst_conf_ideal[m][1]==1:
        no_two_true_inst +=1

        if np.argmax(X_coord_conf_ideal[m])==np.argmax(X_coord_conf_noisy[m][0])==np.argmax(X_coord_conf_noisy[m][1]):
            no_two_good_pred += 1


print(f'Average correct predictions: {(no_good_pred*100/no_true_inst):.2f} %')
print(f'Average correct predictions of both models at the same time: {(no_two_good_pred*100/no_two_true_inst):.2f} %')


Average correct predictions: 95.30 %
Average correct predictions of both models at the same time: 90.95 %


In [92]:
no_good_pred_inst = 0

for group in range(3):
    for m in range(no_instances):
        if X_inst_conf_ideal[m][group] == 1 and X_inst_conf_noisy[m][group] > 0.95:
            no_good_pred_inst += 1
        elif X_inst_conf_ideal[m][group] == 0 and X_inst_conf_noisy[m][group] < 0.05:
            no_good_pred_inst += 1

print(f'{no_good_pred_inst/no_instances/3}')

0.6998916666666667


# Gold Label

Legend:  


0 = NaN (no instruction was given by human, therefore no instruction is forwarded to motion group)  

1 = forward language group's output (human gave language-only-instruction, therefore language-only-instruction overrules other instructions)  

2 to (no_puzzlepieces+2) = puzzle piece ID

In [93]:
for m in range(no_instances):
    
    # language group outputs  
    if X_inst_conf_ideal[m][2]==1:
        y.append(1)
        
    # language and vision group outputs
    elif X_inst_conf_ideal[m][0]==1:
        y.append(np.argmax(X_coord_conf_ideal[m])+2)
        
    # gesture group outputs
    elif X_inst_conf_ideal[m][1]==1:
        y.append(np.argmax(X_coord_conf_ideal[m])+2)
        
    # noone outputs
    else:
        y.append(0)                      
    
    #print(f'instance: {m}')
    #print(y[m])
    #print('\n')


# Store All

In [94]:
X = {'Instruction':X_inst_conf_ideal,'Instruction Confidence':X_inst_conf_noisy, 'Coordinate': X_coord_conf_ideal , 'Coordinate Confidence':X_coord_conf_noisy}
df_X = pd.DataFrame(X)
df_X


,Instruction,Instruction Confidence,Coordinate,Coordinate Confidence
0,"[0, 0, 0]","[0.050103407753331705, 0.04207310414997806, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[[0.08854195637101682, 0.0427370224719794, 0.0..."
1,"[1, 0, 0]","[0.9510318742332541, 0.04853914553666638, 0.04...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.049296963610845505, 0.06187802191620061, 0..."
2,"[1, 1, 0]","[0.9528470552028289, 0.9464943416073138, 0.052...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.07371624055286576, 0.06695540240053945, 0...."
3,"[0, 1, 1]","[0.044857230266676276, 0.9491834801259798, 0.9...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.04474777835817049, 0.0589973861797055, 0.0..."
4,"[0, 0, 1]","[0.04819322716189359, 0.04341594837392296, 0.9...","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.041236578322915864, 0.042444830692797614, ..."
...,...,...,...,...
199995,"[1, 1, 1]","[0.9520212441297283, 0.9526131953720787, 0.947...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[[0.06140178931271305, 0.09837503266530309, 0...."
199996,"[1, 0, 0]","[0.9565887259352829, 0.048357317306439246, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.06383757349139531, 0.0630215659245044, 0.0..."
199997,"[1, 1, 1]","[0.9503105296663972, 0.9513040059669711, 0.945...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0725072156917116, 0.04402003370279818, 0.0..."
199998,"[0, 0, 0]","[0.050066272824196685, 0.0529875476006192, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.06290004860536143, 0.07874905803518725, 0...."


In [95]:
y = {'DM output': y}
df_y = pd.DataFrame(data=y)
df_y

,DM output
0,0
1,12
2,14
3,1
4,1
...,...
199995,1
199996,11
199997,1
199998,0


In [96]:
df_X.to_json(r'/content/drive/MyDrive/HRI/X_DM.json', orient='index')
df_y.to_json(r'/content/drive/MyDrive/HRI/y_DM.json', orient='index')


with open('/content/drive/MyDrive/HRI/X_DM.json') as f:
    data = json.load(f)

df = pd.DataFrame.from_dict(data,orient='index')
df.head()


,Instruction,Instruction Confidence,Coordinate,Coordinate Confidence
0,"[0, 0, 0]","[0.0501034078, 0.0420731041, 0.0492276884]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","[[0.0885419564, 0.0427370225, 0.0850219318, 0...."
1,"[1, 0, 0]","[0.9510318742, 0.0485391455, 0.0445237955]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0492969636, 0.0618780219, 0.0485469618, 0...."
2,"[1, 1, 0]","[0.9528470552, 0.9464943416, 0.052636881]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0737162406, 0.0669554024, 0.0551531139, 0...."
3,"[0, 1, 1]","[0.0448572303, 0.9491834801, 0.9480354619]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0447477784, 0.0589973862, 0.0418077271, 0...."
4,"[0, 0, 1]","[0.0481932272, 0.0434159484, 0.9510766132]","[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0412365783, 0.0424448307, 0.0746511138, 0...."


In [97]:
df_new = df.drop(['Instruction', 'Coordinate'], axis=1)

instr_df = pd.DataFrame(df_new['Instruction Confidence'].to_list())
instr_df.head()

coord_df = pd.DataFrame(df_new['Coordinate Confidence'].to_list())

coord_df0 = pd.DataFrame(coord_df[0].to_list())
coord_df1 = pd.DataFrame(coord_df[1].to_list())

a = [f'Instr{i}' for i in range(3)]
b = [f'LV{i}' for i in range(no_puzzlepieces)]
c = [f'G{i}' for i in range(no_puzzlepieces)]

col_names = []
col_names.extend(a)
col_names.extend(b)
col_names.extend(c)

final_df_noisy = pd.concat([instr_df, coord_df0, coord_df1], axis=1)
final_df_noisy.columns = col_names
final_df_noisy.head()


,Instr0,Instr1,Instr2,LV0,LV1,LV2,LV3,LV4,LV5,LV6,...,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14
0,0.050103,0.042073,0.049228,0.088542,0.042737,0.085022,0.087104,0.051567,0.054595,0.103419,...,0.092731,0.056785,0.060187,0.059378,0.057118,0.052373,0.056950,0.052600,0.058280,0.083575
1,0.951032,0.048539,0.044524,0.049297,0.061878,0.048547,0.068868,0.058018,0.043607,0.053688,...,0.064793,0.087483,0.065898,0.048003,0.043908,0.080280,0.049028,0.058501,0.080955,0.056883
2,0.952847,0.946494,0.052637,0.073716,0.066955,0.055153,0.075863,0.036481,0.054836,0.046503,...,0.072805,0.036467,0.056491,0.080342,0.062255,0.084040,0.053800,0.156755,0.049816,0.051558
3,0.044857,0.949183,0.948035,0.044748,0.058997,0.041808,0.049508,0.085050,0.044038,0.068809,...,0.039506,0.045965,0.050892,0.066380,0.067819,0.078435,0.070112,0.059869,0.104711,0.075084
4,0.048193,0.043416,0.951077,0.041237,0.042445,0.074651,0.074058,0.095972,0.093374,0.043448,...,0.058883,0.084214,0.040308,0.055965,0.087991,0.082222,0.085386,0.058715,0.068793,0.079092


In [98]:
df_new = df.drop(['Instruction Confidence', 'Coordinate Confidence'], axis=1)
df_new.head()

instr_df = pd.DataFrame(df_new['Instruction'].to_list())

coord_df = pd.DataFrame(df_new['Coordinate'].to_list())

a = [f'Instr{i}' for i in range(3)]
b = [f'GoldCoord{i}' for i in range(no_puzzlepieces)]

col_names = []
col_names.extend(a)
col_names.extend(b)

final_df_gold = pd.concat([instr_df, coord_df], axis=1)
final_df_gold.columns = col_names
final_df_gold.head()

,Instr0,Instr1,Instr2,GoldCoord0,GoldCoord1,GoldCoord2,GoldCoord3,GoldCoord4,GoldCoord5,GoldCoord6,GoldCoord7,GoldCoord8,GoldCoord9,GoldCoord10,GoldCoord11,GoldCoord12,GoldCoord13,GoldCoord14
0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
df_y.head()

,DM output
0,0
1,12
2,14
3,1
4,1
